In [1]:
import numpy as np
from src.dynamical_systems.gravitational_waves import GravitationalWaves
from src.density_estimator import DensityEstimator

# set random seed
np.random.seed(0)

## Load Gravitational Waves Posteriors from file

In [2]:
# define gravitational waves model
gw_model = GravitationalWaves()

# load posteriors from file
data_box = gw_model.load_data()

# merge classes 0 and 1
class_idx = np.array([0, 1])
data_box_merged = data_box.merge_classes(class_idx)

# down-sample
class_idx = np.array([2])               # select class with labels 2
return_amounts = np.array([30])         # randomly downsample to 30 examples
data_box_down_sampled = data_box_merged.down_sample_examples(class_idx, return_amounts)

data_box_final = data_box_down_sampled

In [ ]:
# Print meta data for GW
print('# examples total:     ', len(data_box.get_labels()))
print('# examples BNS:       ', sum(data_box.get_labels() == 0))
print('# examples NSBH:      ', sum(data_box.get_labels() == 1))
print('# examples small BBH: ', sum(data_box.get_labels() == 2))
print('# examples large BBH: ', sum(data_box.get_labels() == 3))

In [ ]:
# make cornerplot of posterior samples
num_bins = 100
smoothness = 1
colours = ['blue', 'red', 'green']
labels=[r"$m_{1}$",
        r"$m_{2}$",
        r"$s_{1}$",
        r"$s_{2}$"]
limits = np.array([[0, 100],
                   [0, 100],
                   [-1, 1],
                   [-1, 1],])

title = 'Gravitational Waves'
legend = ['NS - BH / NS - NS', 'Small BH - Small BH', 'Large BH - Large BH ']
figsize = (15, 6)

data_box_final.plot_posterior_samples(num_bins, smoothness, colours, labels, limits, title, legend, figsize)

## Density Estimation

In [ ]:
# create density estimate

# set density estimation options
density_est_opts = {
    'type':             'bayesian_gaussian_mixture',
    'covariance_type':                  'spherical',    # 'full', 'diag' 'spherical' 
    'n_components':                              20,    # 10
    'n_init' :                                   20,    # 15
    'max_iter':                                 500,    # 200
    'trim_percent':                            0.99
}

# define density estimator object
dens_estimator = DensityEstimator(data_box_final, density_est_opts)

# run sampling
data_box_spherical = dens_estimator.run_density_estimation()

## Save data to file

In [5]:
# save data boxes with created density estimates
data_box_spherical.save_data('spherical_finite_mixture')